In [1]:
import numpy as np
import torch
from load_msi import create_msi
from visualization import (
        generate_nx_graph, generate_khop_subgraph, generate_pyvis_graph, 
        visualize_tsne_embeddings, visualize_pca_embeddings)
from training import split_data, create_minibatches, train_link_predictor
from torch_geometric.utils import to_networkx, degree
from torch_geometric.nn import to_hetero

from gnn import GAT, GIN, DotProductLinkPredictor
import torch_geometric.transforms as T
print(torch.__version__)

1.13.1+cu117


In [2]:
msi = create_msi("msi", 
                 num_features = 1,
                 data_dir="../multiscale_interactome/data")
data = msi.hetero_data
data = T.ToUndirected()(data)

In [3]:
# Generate and visualize graphs (HIV: id = C0019693, gid = 840)
##G, G_undirected = generate_nx_graph(msi.data)
##subG = generate_khop_subgraph(msi, G, 840, 1)
##py_graph = generate_pyvis_graph(subG)
##py_graph.show("test.html")

In [4]:
# Set the device.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cuda


In [5]:
edge_type = ("dz", "implicates", "prot")
id_map = msi.mappings["dz_id_gid_map"]
name_map = msi.mappings["dz_id_name_map"]



degrees = degree(data[edge_type].edge_index[0]).numpy()
sorted_gids = np.argsort(degrees)[::-1]
sorted_degs = np.sort(degrees)[::-1]


#for gid, deg in zip(sorted_gids, sorted_degs):
#    id = list(id_map.keys())[list(id_map.values()).index(gid)]
#    name = name_map[id]
#    print(f"Disease: {name:<45}   |   1-hop neighbors: {deg:>5}   |   id: {id}")


In [6]:
# Prepare data for GNN.
edge_types = ("drug", "treats", "dz")
rev_edge_types = ("dz", "rev_treats", "drug")

# Split the data into train, validation, and test sets.
train_data, val_data, test_data = split_data(
        data, val_size=0.1, test_size=0.1, supervision_ratio=0.3,
        edge_types=edge_types, neg_sampling_ratio=1.0,
        rev_edge_types=rev_edge_types)




# Create minibatches.
train_loader = create_minibatches(
        train_data, edge_types=edge_types, num_khop_neighbors=[50,25,10, 10], neg_sampling_ratio=1.0, batch_size=16)
val_loader = create_minibatches(
        val_data, edge_types=edge_types, num_khop_neighbors = [50,25,10, 10], batch_size=16)



In [7]:
# Instantiate the model and generate some initial node embeddings prior to training.
model = GAT(64, 16)
model = to_hetero(model, data.metadata(), aggr='sum').to(device)
predictor = DotProductLinkPredictor(32, 32, 1).to(device)

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
#model.reset_parameters()
predictor.reset_parameters()

# Train and validate the model for a given number of epochs.
for epoch in range(1, 30+1):
    
    # Training..
    loss, auc = train_link_predictor(
            model, predictor, "drug", "dz", edge_types=edge_types, 
            loader=train_loader, optimizer=optimizer, device=device, train=True)
    
    # Validation.
    val_loss, val_auc = train_link_predictor(
            model, predictor, "drug", "dz", edge_types=edge_types, 
            loader=val_loader,  optimizer=optimizer, device=device, train=False)
    
    print(f"Epoch     : {epoch}")
    print(f" Loss     : {loss:<10.4f} |     AUC:     {auc:>10.4f}")
    print(f" Val_Loss : {val_loss:<10.4f} |     Val_AUC: {val_auc:>10.4f}")

100%|██████████| 74/74 [00:15<00:00,  4.89it/s]


Epoch     : 1
 Loss     : 0.8193     |     AUC:         0.5588
 Val_Loss : 0.6444     |     Val_AUC:     0.6831


100%|██████████| 74/74 [00:15<00:00,  4.90it/s]


Epoch     : 2
 Loss     : 0.6480     |     AUC:         0.7032
 Val_Loss : 0.5770     |     Val_AUC:     0.7650


100%|██████████| 74/74 [00:15<00:00,  4.91it/s]


...

Epoch     : 28
 Loss     : 0.4871     |     AUC:         0.8324
 Val_Loss : 0.4994     |     Val_AUC:     0.8380


100%|██████████| 74/74 [00:14<00:00,  4.95it/s]


Epoch     : 29
 Loss     : 0.5013     |     AUC:         0.8149
 Val_Loss : 0.5003     |     Val_AUC:     0.8271


100%|██████████| 74/74 [00:15<00:00,  4.89it/s]

Epoch     : 30
 Loss     : 0.4909     |     AUC:         0.8266
 Val_Loss : 0.5190     |     Val_AUC:     0.8266


In [39]:
msi = create_msi("msi-hiv", 
                 num_features = 1,
                 data_dir="../multiscale_interactome/data", 
                 extra_dz_data="../data/hiv/hiv_protein30.tsv")

hiv_data = msi.hetero_data
hiv_data = T.ToUndirected()(hiv_data)
#(HIV: id = C0019693, gid = 840)
hiv_id = "C0019693"
hiv_gid = 840


In [40]:
data_loader = create_minibatches(
        hiv_data, edge_types=edge_types, num_khop_neighbors = [50,25,10, 10], batch_size=16)

AttributeError: 'EdgeStorage' object has no attribute 'edge_label_index'

In [35]:
def get_node_embeddings(model, data):
    model.eval()
    model.to("cpu")
    data.to("cpu")
    z = model(data.x_dict, data.edge_index_dict)
    return z

In [36]:
torch.cuda.empty_cache()
#print("torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
#print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
#print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))
#print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [37]:
z = get_node_embeddings(model, data)
z_drugs = z["drug"]
hiv_emb = z["dz"][hiv_gid]

RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 1187117568 bytes. Error code 12 (Cannot allocate memory)

In [16]:
from numpy import dot
from numpy.linalg import norm

#hiv_emb = hiv_emb.cpu().detach().numpy()
edge_preds = []
for drug_emb in z_drugs:
    #drug_emb = drug_emb.cpu().detach().numpy()
    #edge_pred = (drug_emb * hiv_emb).sum(dim=-1).cpu().detach().numpy()
    #edge_pred = dot(drug_emb, hiv_emb)/(norm(drug_emb)*norm(hiv_emb))
    edge_pred = predictor(drug_emb, hiv_emb)
    edge_preds.append(edge_pred)
sorted_edges = np.sort(edge_preds)[::-1]
sorted_ind = np.argsort(edge_preds)[::-1]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat2 in method wrapper_mm)

In [ ]:
for ind, edge in zip(sorted_ind, sorted_edges):
    id = [i for i in msi.mappings["drug_id_gid_map"] if msi.mappings["drug_id_gid_map"][i] == ind][0]
    drug = msi.mappings["drug_id_name_map"][id]
    
    print(f"Drug: {drug:<30}    |    HIV edge pred: {edge:>10.4}")